# Final LFM

In [1]:
FACTORS = ?
REGULAR = ?
ITERATIONS = ?
NEG_PROP = ?

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import implicit
import time
from tqdm import tqdm

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\implicit_lfm_123_123_12_12"
score_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\implicit_lfm_123_123_12_12_score"

In [4]:
%%time

list_coord = []
idx = 0

with open(train_path) as f:
    lines = f.readlines()
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
first_id_test = idx

with open(test_path) as f:
    lines = f.readlines()
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
last_id_test = idx - 1

CPU times: total: 14.7 s
Wall time: 14.7 s


In [5]:
first_id_test, last_id_test

(1160084, 1449997)

In [6]:
%%time

rows = [i[0] for i in list_coord]
cols = [i[1] for i in list_coord]
arr_rows = np.array(rows)
arr_cols = np.array(cols)
arr_cols = arr_cols.astype(np.int32)
ones = np.ones(len(list_coord))
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=np.int32)

CPU times: total: 47 s
Wall time: 47.2 s


In [7]:
model = implicit.lmf.LogisticMatrixFactorization(factors=FACTORS,
                                                 learning_rate=1.00,
                                                 regularization=REGULAR,
                                                 iterations=ITERATIONS,
                                                 neg_prop=NEG_PROP,
                                                 random_state=42)

model.fit(user_item)

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [2:05:26<00:00, 501.78s/it]


In [8]:
%%time

userid = np.arange(last_id_test + 1)
recommendations = model.recommend(userid, user_item, N=100, filter_already_liked_items=True)

CPU times: total: 7h 21min 57s
Wall time: 25min 6s


In [9]:
result = [' '.join(map(str, i)) + '\n' for idx, i in tqdm(enumerate(recommendations[0])) if first_id_test <= idx <= last_id_test]
with open(pred_path, 'w') as f:
    f.writelines(result)

1449998it [00:09, 156782.39it/s]


In [10]:
result = [' '.join(map(str, i)) + '\n' for idx, i in tqdm(enumerate(recommendations[1])) if first_id_test <= idx <= last_id_test]
with open(score_path, 'w') as f:
    f.writelines(result)

1449998it [00:07, 190413.09it/s] 
